# Phase 4 — Preprocessing Pipelines

In this phase, preprocessing pipelines are introduced to handle numerical and categorical features in a structured and leakage-safe manner.

The objectives of this phase are:
- incorporate additional relevant features
- apply appropriate preprocessing (imputation, encoding, scaling)
- ensure transformations are learned only from training data
- prepare data for more expressive machine learning models


In [1]:
import sys
from pathlib import Path
ROOT = Path.cwd().parent
SRC = ROOT / "src"
if str(SRC) not in sys.path:
    sys.path.append(str(SRC))

from energy_forecast.io import load_data
from energy_forecast.split import time_split
from energy_forecast.evaluate import root_mean_squared_error

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge


#### **Build preprocessing pipelines**

#### numerical pipeline

In [2]:
df = load_data("../data/Energy Production Dataset.csv", date_col="Date")
train_df, val_df, test_df = time_split(df, time_col="Date")


#### categorical pipeline

In [3]:
TARGET = "Production"

X_train = train_df.drop(columns=[TARGET])
y_train = train_df[TARGET]

X_val = val_df.drop(columns=[TARGET])
y_val = val_df[TARGET]

X_test = test_df.drop(columns=[TARGET])
y_test = test_df[TARGET]


#### **Combine pipelines**

In [4]:
num_cols = X_train.select_dtypes(include="number").columns
cat_cols = X_train.select_dtypes(include="object").columns

num_cols, cat_cols


(Index(['Start_Hour', 'End_Hour', 'Day_of_Year'], dtype='object'),
 Index(['Source', 'Day_Name', 'Month_Name', 'Season'], dtype='object'))

In [5]:
preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ]
)


#### **Model + Pipeline (Ridge)**

In [6]:
model = Pipeline(
    steps=[
        ("preprocess", preprocess),
        ("ridge", Ridge(alpha=1.0, random_state=42)),
    ]
)

model.fit(X_train, y_train)


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocess', ...), ('ridge', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers cont

In [7]:
val_rmse = root_mean_squared_error(y_val, model.predict(X_val))
test_rmse = root_mean_squared_error(y_test, model.predict(X_test))

print("Phase 4 RMSE (with preprocessing)")
print("Val :", val_rmse)
print("Test:", test_rmse)


Phase 4 RMSE (with preprocessing)
Val : 4395.907490210907
Test: 4256.893085031952


**Observation:**  
The preprocessing pipeline with additional categorical features slightly improves validation RMSE compared to the Phase 3 baseline. This indicates that contextual features contribute meaningful predictive signal, while larger performance gains are expected from time-dependent feature engineering in subsequent phases.


## Phase 4 Summary — Preprocessing Pipelines

In this phase, preprocessing pipelines were introduced to systematically handle numerical and categorical features.

Key outcomes:
- Numerical and categorical features were processed using dedicated pipelines.
- Missing values were handled via imputation strategies appropriate to feature type.
- Categorical variables were encoded using one-hot encoding with safe handling of unseen categories.
- A Ridge Regression model was trained using the full preprocessing pipeline.
- The pipeline-based model provides a more realistic and extensible baseline for subsequent feature engineering and advanced models.

This phase establishes a robust foundation for introducing lag features, rolling statistics, and non-linear models in later phases.


---